<a href="https://colab.research.google.com/github/UrologyUnbound/SIOP_ML_2024_Discord/blob/main/colabs/Tune_Clarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rating Item Clarity Model Fine Tuning

The goal of this notebook is to fine tune the OpenAI GPT 3.5 model to be able to predict the average clarity rating for each personality item based on responses.

## Inputs and Setup

In [ ]:
!pip install --upgrade tiktoken openai

In [2]:
import json
import openai
import os
import pandas as pd
from pprint import pprint
import tiktoken
from sklearn.model_selection import train_test_split
from google.colab import userdata
import numpy as np
from collections import defaultdict

client = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
encoding = tiktoken.get_encoding("cl100k_base")

In [3]:
#Estimated token counter
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

## Data Import and Prep

In [24]:
df_clarity = pd.read_csv("https://raw.githubusercontent.com/UrologyUnbound/SIOP_ML_2024_Discord/main/data/train/clarity_train.csv")
df_clarity.clarity = df_clarity.clarity.astype(str)
df_clarity.head()

,_id,personality_item,clarity
0,0,Am considered well-off financially.,3.421052631578947
1,1,Make problems bigger than they are.,6.545454545454546
2,2,Judge people by their appearance.,6.545454545454546
3,3,"Did not feel like eating, even though I should...",3.75
4,4,Feel that very few merchants take advantage of...,5.210526315789473


In [25]:
df_clarity.dtypes

_id                  int64
personality_item    object
clarity             object
dtype: object

In [26]:
df_clarity.iloc[df_clarity.personality_item.str.len().idxmax()].personality_item

'Believe that one needs to show their talents and abilities in order to get opportunities and make progress.'

In [27]:
x_train,x_test,y_train,y_test = train_test_split(df_clarity["personality_item"],df_clarity["clarity"],random_state=42,test_size = 0.2)
train_data = pd.concat([x_train , y_train], axis = 1)
test_data = pd.concat([x_train , y_train], axis = 1)
train_data.head()


,personality_item,clarity
28,Work longer hours than most people.,6.375
24,Am able to work hard to achieve results that I...,3.538461538461538
12,Believe that one needs to show their talents a...,5.25
0,Am considered well-off financially.,3.421052631578947
4,Feel that very few merchants take advantage of...,5.210526315789473


In [29]:
system_message = """
Your task is to predict the average clarity rating for each item based on the responses. Respondents rated the clarity of personality test items using a 7-point scale from 1 = extremely unclear to 7 = extremely clear. The output should not make up information and not reference these given instructions or context; only output the answer.
"""

def create_user_message(row):
    return row['personality_item']


def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})

    messages.append({"role": "assistant", "content": row["clarity"]})

    return {"messages": messages}


pprint(prepare_example_conversation(train_data.iloc[0]))

{'messages': [{'content': '\n'
                          'Your task is to predict the average clarity rating '
                          'for each item based on the responses. Respondents '
                          'rated the clarity of personality test items using a '
                          '7-point scale from 1 = extremely unclear to 7 = '
                          'extremely clear. The output should not make up '
                          'information and not reference these given '
                          'instructions or context; only output the answer.\n',
               'role': 'system'},
              {'content': 'Work longer hours than most people.',
               'role': 'user'},
              {'content': '6.375', 'role': 'assistant'}]}


In [30]:
training_json = train_data.apply(prepare_example_conversation, axis=1).tolist()
test_json = test_data.apply(prepare_example_conversation, axis=1).tolist()


for example in training_json[:5]:
    print(example)

{'messages': [{'role': 'system', 'content': '\nYour task is to predict the average clarity rating for each item based on the responses. Respondents rated the clarity of personality test items using a 7-point scale from 1 = extremely unclear to 7 = extremely clear. The output should not make up information and not reference these given instructions or context; only output the answer.\n'}, {'role': 'user', 'content': 'Work longer hours than most people.'}, {'role': 'assistant', 'content': '6.375'}]}
{'messages': [{'role': 'system', 'content': '\nYour task is to predict the average clarity rating for each item based on the responses. Respondents rated the clarity of personality test items using a 7-point scale from 1 = extremely unclear to 7 = extremely clear. The output should not make up information and not reference these given instructions or context; only output the answer.\n'}, {'role': 'user', 'content': 'Am able to work hard to achieve results that I will only get at a time far in

In [33]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)


In [34]:
training_file_name = "tmp_clarity_finetune_training.jsonl"
write_jsonl(training_json, training_file_name)

testing_file_name = "tmp_clarity_finetune_testing.jsonl"
write_jsonl(test_json, testing_file_name)

In [35]:
!head -n 5 tmp_clarity_finetune_training.jsonl

{"messages": [{"role": "system", "content": "\nYour task is to predict the average clarity rating for each item based on the responses. Respondents rated the clarity of personality test items using a 7-point scale from 1 = extremely unclear to 7 = extremely clear. The output should not make up information and not reference these given instructions or context; only output the answer.\n"}, {"role": "user", "content": "Work longer hours than most people."}, {"role": "assistant", "content": "6.375"}]}
{"messages": [{"role": "system", "content": "\nYour task is to predict the average clarity rating for each item based on the responses. Respondents rated the clarity of personality test items using a 7-point scale from 1 = extremely unclear to 7 = extremely clear. The output should not make up information and not reference these given instructions or context; only output the answer.\n"}, {"role": "user", "content": "Am able to work hard to achieve results that I will only get at a time far in

### Pre-Tuning Checks

In [36]:
# Format error checks - Training set
with open("/content/tmp_clarity_finetune_testing.jsonl", 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [37]:
# Format error checks - Training set
with open("/content/tmp_clarity_finetune_training.jsonl", 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [38]:
# Warnings and tokens counts
with open("/content/tmp_clarity_finetune_testing.jsonl", 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 90, 110
mean / median: 96.25, 95.5
p5 / p95: 91.3, 103.5

#### Distribution of num_assistant_tokens_per_example:
min / max: 3, 7
mean / median: 5.375, 7.0
p5 / p95: 3.0, 7.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [39]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")
print(f"Estimated training cost ~${((n_epochs * n_billing_tokens_in_dataset)/1000)*.0080}")

Dataset has ~2310 tokens that will be charged for during training
By default, you'll train for 4 epochs on this dataset
By default, you'll be charged for ~9240 tokens
Estimated training cost ~$0.07392


In [40]:
with open(training_file_name, "rb") as training_fd:
    training_response = client.files.create(
        file=training_fd, purpose="fine-tune"
    )

training_file_id = training_response.id

with open(testing_file_name, "rb") as validation_fd:
    validation_response = client.files.create(
        file=validation_fd, purpose="fine-tune"
    )
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-npvKJ96j1Ol0S1oyXYt6OPt3
Validation file ID: file-RlYxbVyE3v1EiQF3WPPEOONM


## Fine Tuning

In [41]:
# Only Run this cell when wanting to create a new fine-tuning job, otherwise you will be paying to redo work

#Uncomment the below code when wanting to run a new fine-tuning job
# response = client.fine_tuning.jobs.create(
#     training_file=training_file_id,
#     validation_file=validation_file_id,
#     model="gpt-3.5-turbo",
#     suffix="clarity_tuned_v1",
# )

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-nZm5N643P9pHXqzQphu5KrwE
Status: validating_files


In [46]:
#Check Job Status
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)


Job ID: ftjob-nZm5N643P9pHXqzQphu5KrwE
Status: succeeded
Trained Tokens: 9048


In [47]:
#Track Fine-Tuning Endpoints
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 79/96: training loss=1.02, validation loss=0.68
Step 80/96: training loss=1.08, validation loss=0.77
Step 81/96: training loss=1.20, validation loss=0.85
Step 82/96: training loss=0.77, validation loss=0.49
Step 83/96: training loss=0.93, validation loss=0.75
Step 84/96: training loss=0.91, validation loss=0.79
Step 85/96: training loss=0.68, validation loss=0.95
Step 86/96: training loss=0.90, validation loss=0.90
Step 87/96: training loss=0.95, validation loss=0.95
Step 88/96: training loss=0.97, validation loss=0.55
Step 89/96: training loss=0.76, validation loss=1.36
Step 90/96: training loss=0.84, validation loss=0.68
Step 91/96: training loss=0.68, validation loss=0.67
Step 92/96: training loss=1.37, validation loss=0.69
Step 93/96: training loss=0.49, validation loss=1.18
Step 94/96: training loss=0.71, validation loss=0.89
Step 95/96: training loss=0.95, validation loss=1.06
Step 96/96: training loss=0.54, validation loss=0.52
New fine-tuned model created: ft:gpt-3.5-turbo

In [48]:
# When job is done, run to fets fine-tuned model id
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:clarity-tuned-v1:98vb9W4i


## Fine-Tuned Model Testing

In [49]:
df_clarity_dev = pd.read_csv("https://raw.githubusercontent.com/UrologyUnbound/SIOP_ML_2024_Discord/main/data/dev/clarity_val_public.csv")

In [50]:
test_row = df_clarity_dev.iloc[2]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = create_user_message(test_row)
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)

[{'content': '\n'
             'Your task is to predict the average clarity rating for each item '
             'based on the responses. Respondents rated the clarity of '
             'personality test items using a 7-point scale from 1 = extremely '
             'unclear to 7 = extremely clear. The output should not make up '
             'information and not reference these given instructions or '
             'context; only output the answer.\n',
  'role': 'system'},
 {'content': 'Have no sympathy for rule-breakers.', 'role': 'user'}]


In [51]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response.choices[0].message.content)

5.0
